## 1. Setup - Colab Kontrolü ve Proje Kurulumu

In [ ]:
import sys
import os

# Colab kontrolü
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")

    # GPU kontrolü
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")

    # GitHub'dan projeyi klonla
    print("\n📥 Proje indiriliyor...")
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git

    # Proje dizinine geç
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")

    # Gerekli paketleri yükle (sadece temel ML paketleri, Jupyter paketleri hariç)
    print("\n📦 Paketler yükleniyor...")
    !pip install -q torch transformers datasets scikit-learn pandas tqdm wandb matplotlib seaborn
    print("✅ Tüm paketler yüklendi")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

## 2. Import Modüller

In [ ]:
import torch
import numpy as np
import json
from datetime import datetime

from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model

print("✅ Modüller yüklendi")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Konfigürasyon - Focal Loss Enabled

In [ ]:
# Training parametreleri - FOCAL LOSS VERSION
CONFIG = {
    'model_name': 'bert-base-uncased',
    'max_length': 512,
    'batch_size': 16,
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'warmup_steps': 500,
    'threshold': 0.5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'output_dir': './outputs',
    
    # 🎯 FOCAL LOSS PARAMETERS
    'use_focal_loss': True,
    'focal_alpha': 0.25,  # Weight for positive class (0.25 = focus on positives)
    'focal_gamma': 2.0,   # Focusing parameter (2.0 = standard, higher = more aggressive)
}

# Konfigürasyonu yazdır
print("\n" + "="*70)
print("🎯 FOCAL LOSS TRAINING CONFIGURATION")
print("="*70)
for key, value in CONFIG.items():
    marker = "🎯" if 'focal' in key else "  "
    print(f"{marker} {key:20s}: {value}")
print("="*70)
print("\n💡 Focal Loss formula: FL(p_t) = -α(1-p_t)^γ * log(p_t)")
print(f"   α (alpha) = {CONFIG['focal_alpha']} : Balances positive/negative class importance")
print(f"   γ (gamma) = {CONFIG['focal_gamma']} : Down-weights easy examples (higher = more focus on hard examples)")
print("="*70)

## 4. Veri Yükleme ve Hazırlama

In [ ]:
print("\n" + "="*70)
print("STEP 1: DATA PREPARATION")
print("="*70 + "\n")

data = prepare_data(
    model_name=CONFIG['model_name'],
    max_length=CONFIG['max_length']
)

train_dataset = data['train_dataset']
test_dataset = data['test_dataset']
label_list = data['label_list']
num_labels = data['num_labels']

print(f"\n✅ Veri hazırlama tamamlandı!")
print(f"   Train samples: {len(train_dataset)}")
print(f"   Test samples: {len(test_dataset)}")
print(f"   Number of labels: {num_labels}")

# Class imbalance analizi
all_train_labels = torch.stack([train_dataset[i]['labels'] for i in range(len(train_dataset))])
positive_ratio = all_train_labels.float().mean().item()
print(f"\n⚠️  Class Imbalance:")
print(f"   Positive label ratio: {positive_ratio:.4f} ({positive_ratio*100:.2f}%)")
print(f"   Imbalance ratio: 1:{int(1/positive_ratio)}")
print(f"   This is why we need Focal Loss!")

## 5. Model Yükleme - With Focal Loss

In [ ]:
print("\n" + "="*70)
print("STEP 2: MODEL INITIALIZATION WITH FOCAL LOSS")
print("="*70 + "\n")

model = load_model(
    model_name=CONFIG['model_name'],
    num_labels=num_labels,
    device=CONFIG['device'],
    use_focal_loss=CONFIG['use_focal_loss'],
    focal_alpha=CONFIG['focal_alpha'],
    focal_gamma=CONFIG['focal_gamma']
)

print(f"\n✅ Model yüklendi ve {CONFIG['device']} cihazına taşındı!")
print(f"🎯 Loss function: Focal Loss (α={CONFIG['focal_alpha']}, γ={CONFIG['focal_gamma']})")

## 6. Model Eğitimi

In [ ]:
print("\n" + "="*70)
print("STEP 3: MODEL TRAINING WITH FOCAL LOSS")
print("="*70 + "\n")

# Output dizini oluştur
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"{CONFIG['model_name'].replace('/', '_')}_focal_loss_{timestamp}"
output_dir = os.path.join(CONFIG['output_dir'], run_name)
os.makedirs(output_dir, exist_ok=True)

# Label listesini kaydet
label_file = os.path.join(output_dir, "labels.json")
with open(label_file, 'w') as f:
    json.dump(label_list, f, indent=2)

# Config'i kaydet
config_file = os.path.join(output_dir, "config.json")
with open(config_file, 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Eğitimi başlat
history = train_model(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    output_dir=output_dir,
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    num_epochs=CONFIG['num_epochs'],
    warmup_steps=CONFIG['warmup_steps'],
    device=CONFIG['device']
)

# Training geçmişini kaydet
history_file = os.path.join(output_dir, "training_history.json")
with open(history_file, 'w') as f:
    json.dump(history, f, indent=2)

print(f"\n✅ Eğitim tamamlandı!")
print(f"   Final train loss: {history['train_loss'][-1]:.4f}")
if 'val_loss' in history:
    print(f"   Final val loss: {history['val_loss'][-1]:.4f}")

# Loss grafiği
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss (Focal)', linewidth=2)
if 'val_loss' in history:
    plt.plot(history['val_loss'], label='Val Loss (Focal)', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Focal Loss')
plt.title('Training History - Focal Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(output_dir, 'training_loss.png'))
plt.show()

print(f"\n📊 Loss grafiği kaydedildi: training_loss.png")

## 7. Model Değerlendirme - Prediction Analysis

In [ ]:
print("\n" + "="*70)
print("STEP 4: MODEL EVALUATION - FOCAL LOSS")
print("="*70 + "\n")

# Model çıktılarını kontrol et - Focal Loss ile iyileşme bekleniyor
print("🔍 Focal Loss ile model çıktılarını kontrol ediyorum...")
model.eval()
sample_batch = next(iter(torch.utils.data.DataLoader(test_dataset, batch_size=16)))
with torch.no_grad():
    sample_out = model(
        input_ids=sample_batch['input_ids'].to(CONFIG['device']),
        attention_mask=sample_batch['attention_mask'].to(CONFIG['device'])
    )
    sample_probs = torch.sigmoid(sample_out['logits'])

print(f"\nSample sigmoid outputs (Focal Loss):")
print(f"  Min: {sample_probs.min().item():.6f}")
print(f"  Max: {sample_probs.max().item():.6f}")
print(f"  Mean: {sample_probs.mean().item():.6f}")
print(f"  Median: {sample_probs.median().item():.6f}")
print(f"  Std: {sample_probs.std().item():.6f}")

print(f"\n💡 Expected improvement over BCE:")
print(f"   - Higher max values (more confident predictions)")
print(f"   - Higher variance (not all predictions near 0)")
print(f"   - Better calibration for rare labels")

# Otomatik threshold belirleme
optimal_threshold = float(sample_probs.median().item())
print(f"\n💡 Önerilen threshold: {optimal_threshold:.4f}")

# Farklı threshold değerleri ile değerlendir
thresholds_to_test = [0.1, 0.2, 0.3, optimal_threshold, 0.5]
print(f"\n📊 Farklı threshold değerleri test ediliyor...")

best_f1 = 0
best_threshold = 0.5
best_metrics = None

for thresh in thresholds_to_test:
    metrics = evaluate_model(
        model=model,
        test_dataset=test_dataset,
        batch_size=CONFIG['batch_size'],
        device=CONFIG['device'],
        threshold=thresh,
        label_list=label_list
    )

    if metrics['micro_f1'] > best_f1:
        best_f1 = metrics['micro_f1']
        best_threshold = thresh
        best_metrics = metrics

print(f"\n🏆 En iyi threshold: {best_threshold:.4f}")
print(f"   Micro F1: {best_f1:.4f}")

# En iyi metrikleri kaydet
metrics_to_save = {k: float(v) if isinstance(v, (float, int)) else v
                   for k, v in best_metrics.items()
                   if k not in ['predictions', 'labels']}
metrics_to_save['best_threshold'] = best_threshold
metrics_to_save['loss_type'] = 'focal_loss'
metrics_to_save['focal_alpha'] = CONFIG['focal_alpha']
metrics_to_save['focal_gamma'] = CONFIG['focal_gamma']

metrics_file = os.path.join(output_dir, "evaluation_metrics.json")
with open(metrics_file, 'w') as f:
    json.dump(metrics_to_save, f, indent=2)

print(f"\n✅ Değerlendirme tamamlandı!")
print(f"\nFinal Metrics (Focal Loss, threshold={best_threshold:.4f}):")
print(f"  Micro F1:    {best_metrics['micro_f1']:.4f}")
print(f"  Macro F1:    {best_metrics['macro_f1']:.4f}")
print(f"  Samples F1:  {best_metrics['samples_f1']:.4f}")
print(f"  Precision:   {best_metrics['micro_precision']:.4f}")
print(f"  Recall:      {best_metrics['micro_recall']:.4f}")

## 8. 🔬 Top-K Strategy Evaluation

In [ ]:
print("\n" + "="*70)
print("🔬 TOP-K EVALUATION WITH FOCAL LOSS")
print("="*70 + "\n")

# Calculate average number of true labels per sample
true_labels_sum = torch.stack([test_dataset[i]['labels'] for i in range(len(test_dataset))]).sum(dim=1)
avg_k = int(true_labels_sum.float().mean().item())
print(f"Average true labels per sample: {avg_k:.1f}")

# Test different K values
k_values = [1, 3, 5, avg_k, 10, 15, 20]
topk_results = []

for k in k_values:
    print(f"\nTesting Top-K with k={k}...")
    
    model.eval()
    all_predictions = []
    all_labels = []
    
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    with torch.no_grad():
        for batch in test_loader:
            outputs = model(
                input_ids=batch['input_ids'].to(CONFIG['device']),
                attention_mask=batch['attention_mask'].to(CONFIG['device'])
            )
            probs = torch.sigmoid(outputs['logits'])
            
            # Select top-k
            batch_preds = torch.zeros_like(probs)
            topk_values, topk_indices = torch.topk(probs, k=min(k, probs.size(1)), dim=1)
            batch_preds.scatter_(1, topk_indices, 1.0)
            
            all_predictions.append(batch_preds.cpu().numpy())
            all_labels.append(batch['labels'].cpu().numpy())
    
    predictions = np.vstack(all_predictions)
    labels = np.vstack(all_labels)
    
    # Compute metrics
    from sklearn.metrics import f1_score, precision_score, recall_score
    
    micro_f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    macro_f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    samples_f1 = f1_score(labels, predictions, average='samples', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)
    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    
    topk_results.append({
        'k': k,
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
        'samples_f1': samples_f1,
        'precision': precision,
        'recall': recall
    })
    
    print(f"  Micro F1: {micro_f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

# Display results table
print(f"\n{'='*80}")
print("🎯 FOCAL LOSS - TOP-K STRATEGY RESULTS")
print(f"{'='*80}\n")
print(f"{'K':>5} {'Micro F1':>10} {'Macro F1':>10} {'Samples F1':>12} {'Precision':>11} {'Recall':>10}")
print("-" * 80)
for result in topk_results:
    print(f"{result['k']:>5} {result['micro_f1']:>10.4f} {result['macro_f1']:>10.4f} "
          f"{result['samples_f1']:>12.4f} {result['precision']:>11.4f} {result['recall']:>10.4f}")

# Find best K
best_result = max(topk_results, key=lambda x: x['micro_f1'])
print(f"\n🏆 Best K: {best_result['k']} with Micro F1: {best_result['micro_f1']:.4f}")

# Save Top-K results
topk_file = os.path.join(output_dir, "topk_results.json")
with open(topk_file, 'w') as f:
    json.dump(topk_results, f, indent=2)

# Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot F1 scores
k_vals = [r['k'] for r in topk_results]
axes[0].plot(k_vals, [r['micro_f1'] for r in topk_results], marker='o', label='Micro F1', linewidth=2)
axes[0].plot(k_vals, [r['macro_f1'] for r in topk_results], marker='s', label='Macro F1', linewidth=2)
axes[0].plot(k_vals, [r['samples_f1'] for r in topk_results], marker='^', label='Samples F1', linewidth=2)
axes[0].axvline(x=best_result['k'], color='r', linestyle='--', alpha=0.5, label=f'Best K={best_result["k"]}')
axes[0].set_xlabel('K')
axes[0].set_ylabel('F1 Score')
axes[0].set_title('Focal Loss: F1 Scores vs K')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot Precision-Recall
axes[1].plot(k_vals, [r['precision'] for r in topk_results], marker='o', label='Precision', linewidth=2)
axes[1].plot(k_vals, [r['recall'] for r in topk_results], marker='s', label='Recall', linewidth=2)
axes[1].axvline(x=best_result['k'], color='r', linestyle='--', alpha=0.5, label=f'Best K={best_result["k"]}')
axes[1].set_xlabel('K')
axes[1].set_ylabel('Score')
axes[1].set_title('Focal Loss: Precision & Recall vs K')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'topk_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Top-K analiz grafiği kaydedildi: topk_analysis.png")

## 9. 📊 BCE vs Focal Loss Comparison

In [ ]:
print("\n" + "="*80)
print("📊 FOCAL LOSS vs BCE COMPARISON")
print("="*80)

# You can manually enter BCE results from previous run for comparison
print("\n💡 To compare with BCE results:")
print("   1. Check outputs/bert-base-uncased_YYYYMMDD_HHMMSS/evaluation_metrics.json")
print("   2. Compare Micro F1, Macro F1, and prediction statistics")
print("\n📋 Focal Loss Results (this run):")
print(f"   Best threshold: {best_threshold:.4f}")
print(f"   Micro F1: {best_metrics['micro_f1']:.4f}")
print(f"   Macro F1: {best_metrics['macro_f1']:.4f}")
print(f"   Best Top-K: k={best_result['k']}, F1={best_result['micro_f1']:.4f}")

print("\n💡 Expected improvements:")
print("   ✅ Higher F1 scores (especially for rare labels)")
print("   ✅ Better prediction calibration")
print("   ✅ More variance in predictions")
print("   ✅ Reduced false negatives for minority classes")

# Quick variance check
all_test_probs = []
model.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
with torch.no_grad():
    for batch in test_loader:
        outputs = model(
            input_ids=batch['input_ids'].to(CONFIG['device']),
            attention_mask=batch['attention_mask'].to(CONFIG['device'])
        )
        probs = torch.sigmoid(outputs['logits'])
        all_test_probs.append(probs.cpu())

all_test_probs = torch.cat(all_test_probs, dim=0)
variance_per_label = all_test_probs.var(dim=0)
low_variance_count = (variance_per_label < 0.0001).sum().item()

print(f"\n🔍 Prediction Variance Analysis:")
print(f"   Labels with very low variance (<0.0001): {low_variance_count}/{num_labels}")
print(f"   ({(low_variance_count/num_labels)*100:.1f}% of labels)")
print(f"\n   BCE baseline had: 499/499 (100%) - all predictions identical!")
print(f"   Focal Loss improvement: {499 - low_variance_count} labels now have variance!")

## 10. Sonuçları Kaydet ve İndir

In [ ]:
# Özet dosyası oluştur
summary = {
    'model': CONFIG['model_name'],
    'timestamp': timestamp,
    'loss_function': 'Focal Loss',
    'focal_params': {
        'alpha': CONFIG['focal_alpha'],
        'gamma': CONFIG['focal_gamma']
    },
    'configuration': CONFIG,
    'data': {
        'num_labels': num_labels,
        'train_samples': len(train_dataset),
        'test_samples': len(test_dataset),
        'class_imbalance_ratio': f"1:{int(1/positive_ratio)}"
    },
    'training': {
        'final_loss': history['train_loss'][-1],
        'epochs': CONFIG['num_epochs']
    },
    'evaluation': {
        'threshold_based': metrics_to_save,
        'topk_based': {
            'best_k': best_result['k'],
            'best_micro_f1': best_result['micro_f1'],
            'all_k_results': topk_results
        },
        'prediction_stats': {
            'low_variance_labels': low_variance_count,
            'total_labels': num_labels,
            'variance_improvement': f"{499 - low_variance_count} labels gained variance"
        }
    }
}

summary_file = os.path.join(output_dir, "summary.json")
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*70)
print("🎯 FOCAL LOSS PIPELINE COMPLETE!")
print("="*70)
print(f"\nSonuçlar kaydedildi: {output_dir}")
print("\nDosyalar:")
print(f"  - config.json (Focal Loss parameters)")
print(f"  - labels.json")
print(f"  - training_history.json")
print(f"  - evaluation_metrics.json")
print(f"  - topk_results.json")
print(f"  - summary.json")
print(f"  - final_model.pt")
print(f"  - checkpoint_epoch_*.pt")
print(f"  - training_loss.png")
print(f"  - topk_analysis.png")
print("="*70)

## 11. Colab'da Sonuçları İndir

In [ ]:
if IN_COLAB:
    import shutil
    from google.colab import files

    # Sonuçları ZIP'le
    zip_name = f"{run_name}.zip"
    shutil.make_archive(run_name, 'zip', output_dir)

    print(f"\n📦 Sonuçlar sıkıştırılıyor: {zip_name}")
    print(f"   Boyut: {os.path.getsize(zip_name) / (1024*1024):.2f} MB")

    # İndir
    print("\n⬇️  İndirme başlatılıyor...")
    files.download(zip_name)
    print("✅ İndirme tamamlandı!")
else:
    print("ℹ️  Yerel ortamdasınız, sonuçlar zaten bilgisayarınızda.")

## 12. (Opsiyonel) Focal Loss Parameters Tuning

You can experiment with different Focal Loss parameters to find optimal settings for your dataset.

In [ ]:
# Experiment with different Focal Loss parameters:

# More aggressive focusing (harder examples get more weight):
# CONFIG['focal_gamma'] = 3.0  # or 4.0

# More balanced class weighting:
# CONFIG['focal_alpha'] = 0.5  # Equal weight to positive/negative

# Focus more on positive class:
# CONFIG['focal_alpha'] = 0.75  # More weight to positives

# Less aggressive (closer to BCE):
# CONFIG['focal_gamma'] = 1.0

# Then re-run cells from "5. Model Yükleme" onwards
print("💡 Adjust CONFIG['focal_alpha'] and CONFIG['focal_gamma'] above")
print("   Then re-run from Cell 5 to experiment with different parameters")